In [22]:
import pandas as pd
import numpy as np
%config InlineBackend.figure_format = 'retina'
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import scipy
from matplotlib.colors import ListedColormap
from functools import partial
from sklearn.model_selection import KFold
import numpy as np
from sklearn.preprocessing import StandardScaler

In [23]:
df = pd.read_csv("penguins_size.csv")
df = df.drop(["island","culmen_depth_mm","flipper_length_mm"],axis = 1)
df = df[df['species'] == 'Adelie']
df = df.drop(["species"],axis = 1)
df = df.dropna()
df['sex'] = df['sex'].replace({'FEMALE': 1, 'MALE': -1})
df['sex'] = df['sex'].astype(float)
X = df.iloc[:, :-1].values
scaler = StandardScaler()
X = scaler.fit_transform(X)
Y = df.iloc[:, -1].values  
Y = Y.reshape(-1, 1).astype(float) 
#Y[Y == 0] = -1 
X_and_Y = np.hstack((X, Y))     # Stack them together for shuffling.
np.random.seed(1)               # Set the random seed.
np.random.shuffle(X_and_Y)      # Shuffle the data points in X_and_Y array

/tmp/ipykernel_7193/3340130666.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sex'] = df['sex'].replace({'FEMALE': 1, 'MALE': -1})


In [24]:
# Euclidean distance.
def calc_distance(x1, x2):
    # x1, x2 are two NumPy vectors
    # Return the Euclidean distance between x1 and x2. It should be a scalar.
    dist = np.sqrt(np.sum((x1-x2)**2))
    return dist

# k nearest neighbor predictor.
def f_knn(x, X_train, Y_train, k):
    # Create the list of (distance, label) pairs.
    dist_label_pairs = []
    for xi, yi in zip(X_train, Y_train):
        # Calculate the distance.
        dist = calc_distance(xi, x)
        # Add a (distance, label) pair to the list.
        dist_label_pairs.append((dist, yi))
    # Sort the pairs by distance (ascending).
    sorted_dist_label_pairs = sorted(dist_label_pairs, key=lambda x:x[0])
    # Obtain the first k pairs (corresponding to k smallest distances).
    k_dist_label_pairs = sorted_dist_label_pairs[:k]
    # Extract the labels of the k pairs.
    k_labels = [label for _, label in k_dist_label_pairs]
    # Count the number of +1 predictions and -1 predictions.
    pos1_in_k_labels = 0
    neg1_in_k_labels = 0
    for label in k_labels:
        if label == +1:
            pos1_in_k_labels += 1
        elif label == -1:
            neg1_in_k_labels += 1
    # Make the prediction based on counts.
    if pos1_in_k_labels > neg1_in_k_labels:
        y_pred = +1
    else:
        y_pred = -1

    return y_pred

# Judge function: 1(a != b). It supports scalar, vector and matrix.
def judge(a, b):
    return np.array(a != b).astype(np.float32)

# Calculate error given feature vectors X and labels Y.
def calc_error(X, Y, knn_classifier):
    e = 0
    n = len(X)
    for (xi, yi) in zip(X, Y):
        e += judge(yi, knn_classifier(xi))
    e = 1.0 * e / n
    return e

# KNN
# 20 training

In [25]:
# Divide the data points into training, validation, and test sets
X_shuffled = X_and_Y[:, :2]  # First two columns as features
Y_shuffled = X_and_Y[:, 2]   # Third column as target

# Compute split indices
train_split = int(0.2 * len(X_shuffled))  # 20% for training
val_split = train_split + int(0.1 * len(X_shuffled))  # Additional 20% for validation

# Create the splits
X_train = X_shuffled[:train_split]
Y_train = Y_shuffled[:train_split]
X_val = X_shuffled[train_split:val_split]
Y_val = Y_shuffled[train_split:val_split]
X_test = X_shuffled[val_split:]
Y_test = Y_shuffled[val_split:]

# Print shapes
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)
print(Y_test.shape)


(29, 2)
(29,)
(14, 2)
(14,)
(103, 2)
(103,)


In [26]:
opt_val_error = 1.0
opt_k = None

# Try different k(s).
for k in [1,2,3,4,5,6,7,8]:
    # Visualize
    #   1. Validation set (as points).
    #   2. Decision boundary from training set (as background).
    print("k={}".format(k))
    # Create a k-NN classifier with training set.
    knn_classifier = partial(f_knn, X_train=X_train, Y_train=Y_train, k=k)
    # Visualization.
    #vis(X_train, Y_train, knn_classifier)
    # Calculate validation error.
    val_error = calc_error(X_val, Y_val, knn_classifier)
    print("Validation error: {}\n".format(val_error))
    if val_error < opt_val_error:
        opt_val_error = val_error
        opt_k = k
        opt_knn_classifier = knn_classifier
print("Best k={}".format(opt_k))
train_error = calc_error(X_train,Y_train,opt_knn_classifier)
print("Train error:{}".format(train_error))
test_error = calc_error(X_test, Y_test, opt_knn_classifier)
#vis(X_test, Y_test, knn_classifier)
print("Test error: {}".format(test_error))

k=1
Validation error: 0.21428571428571427

k=2
Validation error: 0.21428571428571427

k=3
Validation error: 0.21428571428571427

k=4
Validation error: 0.21428571428571427

k=5
Validation error: 0.14285714285714285

k=6
Validation error: 0.21428571428571427

k=7
Validation error: 0.14285714285714285

k=8
Validation error: 0.21428571428571427

Best k=5
Train error:0.10344827586206896
Test error: 0.13592233009708737


# 50 training

In [27]:
# Divide the data points into training, validation, and test sets
X_shuffled = X_and_Y[:, :2]  # First two columns as features
Y_shuffled = X_and_Y[:, 2]   # Third column as target

# Compute split indices
train_split = int(0.5 * len(X_shuffled))  # 20% for training
val_split = train_split + int(0.1 * len(X_shuffled))  # Additional 20% for validation

# Create the splits
X_train = X_shuffled[:train_split]
Y_train = Y_shuffled[:train_split]
X_val = X_shuffled[train_split:val_split]
Y_val = Y_shuffled[train_split:val_split]
X_test = X_shuffled[val_split:]
Y_test = Y_shuffled[val_split:]

# Print shapes
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)
print(Y_test.shape)


(73, 2)
(73,)
(14, 2)
(14,)
(59, 2)
(59,)


In [28]:
opt_val_error = 1.0
opt_k = None

# Try different k(s).
for k in [1,2,3,4,5,6,7,8]:
    # Visualize
    #   1. Validation set (as points).
    #   2. Decision boundary from training set (as background).
    print("k={}".format(k))
    # Create a k-NN classifier with training set.
    knn_classifier = partial(f_knn, X_train=X_train, Y_train=Y_train, k=k)
    # Visualization.
    #vis(X_train, Y_train, knn_classifier)
    # Calculate validation error.
    val_error = calc_error(X_val, Y_val, knn_classifier)
    print("Validation error: {}\n".format(val_error))
    if val_error < opt_val_error:
        opt_val_error = val_error
        opt_k = k
        opt_knn_classifier = knn_classifier
print("Best k={}".format(opt_k))
train_error = calc_error(X_train,Y_train,opt_knn_classifier)
print("Train error:{}".format(train_error))
test_error = calc_error(X_test, Y_test, opt_knn_classifier)
#vis(X_test, Y_test, knn_classifier)
print("Test error: {}".format(test_error))

k=1
Validation error: 0.07142857142857142

k=2
Validation error: 0.07142857142857142

k=3
Validation error: 0.07142857142857142

k=4
Validation error: 0.07142857142857142

k=5
Validation error: 0.07142857142857142

k=6
Validation error: 0.07142857142857142

k=7
Validation error: 0.07142857142857142

k=8
Validation error: 0.07142857142857142

Best k=1
Train error:0.0
Test error: 0.06779661016949153


# 80 training

In [29]:
# Divide the data points into training, validation, and test sets
X_shuffled = X_and_Y[:, :2]  # First two columns as features
Y_shuffled = X_and_Y[:, 2]   # Third column as target

# Compute split indices
train_split = int(0.8 * len(X_shuffled))  # 20% for training
val_split = train_split + int(0.1 * len(X_shuffled))  # Additional 20% for validation

# Create the splits
X_train = X_shuffled[:train_split]
Y_train = Y_shuffled[:train_split]
X_val = X_shuffled[train_split:val_split]
Y_val = Y_shuffled[train_split:val_split]
X_test = X_shuffled[val_split:]
Y_test = Y_shuffled[val_split:]

# Print shapes
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)
print(Y_test.shape)


(116, 2)
(116,)
(14, 2)
(14,)
(16, 2)
(16,)


In [30]:
opt_val_error = 1.0
opt_k = None

# Try different k(s).
for k in [1,2,3,4,5,6,7,8]:
    # Visualize
    #   1. Validation set (as points).
    #   2. Decision boundary from training set (as background).
    print("k={}".format(k))
    # Create a k-NN classifier with training set.
    knn_classifier = partial(f_knn, X_train=X_train, Y_train=Y_train, k=k)
    # Visualization.
    #vis(X_train, Y_train, knn_classifier)
    # Calculate validation error.
    val_error = calc_error(X_val, Y_val, knn_classifier)
    print("Validation error: {}\n".format(val_error))
    if val_error < opt_val_error:
        opt_val_error = val_error
        opt_k = k
        opt_knn_classifier = knn_classifier
print("Best k={}".format(opt_k))
train_error = calc_error(X_train,Y_train,opt_knn_classifier)
print("Train error:{}".format(train_error))
test_error = calc_error(X_test, Y_test, opt_knn_classifier)
#vis(X_test, Y_test, knn_classifier)
print("Test error: {}".format(test_error))

k=1
Validation error: 0.07142857142857142

k=2
Validation error: 0.14285714285714285

k=3
Validation error: 0.2857142857142857

k=4
Validation error: 0.2857142857142857

k=5
Validation error: 0.2857142857142857

k=6
Validation error: 0.21428571428571427

k=7
Validation error: 0.21428571428571427

k=8
Validation error: 0.2857142857142857

Best k=1
Train error:0.0
Test error: 0.0625
